# Lombardi Number
Let's define a team's Lombardi number as the shortest chain it takes for them to beat the SB champ. Ex: in the 2019 season, DAL's is 3 (DAL > MIA > IND > KC). The lower the number, the better.

As an extension of this idea, let's define a team's Jackson number as the shortest chain it takes for them to lose to the worst team (i.e. #1 pick). Ex: in the 2020 season, the Packers have a Jackson number of 2 (GB < IND < JAX). Note: this is named after Hue Jackson, who finished with the worst team two years in a row in recent history.

Of course, this program isn't meant to evaluate well/poorly a team performed in a given season, since much of this depends on differences in scheduling. This is mostly an academic exercise for me to practice object-oriented programming, data structures and algorithms (this program uses queues to conduct a breadth-first search through a tree-like search space). 

Enjoy!

In [1]:
import pandas as pd
import numpy as np

In [2]:
class Team():
    def __init__(self, name, year = 2020):
        self.name = name
        self.year = year
        self.losses = list(games[self.year].loc[(games[self.year]['Loser/tie'] == self.name) & (games[self.year]['PtsW'] != games[self.year]['PtsL'])]['Winner/tie'])
        self.wins = list(games[self.year].loc[(games[self.year]['Winner/tie'] == self.name) & (games[self.year]['PtsW'] != games[self.year]['PtsL'])]['Loser/tie'])
        if len(self.losses) + len(self.wins) == 16:
            self.record = str(len(self.wins)) + '-' + str(len(self.losses))
        else:
            self.record = str(len(self.wins)) + '-' + str(len(self.losses)) + '-' + str(16 - len(self.wins) - len(self.losses))
    
    def get_name(self):
        return self.name
    
    def get_losses(self):
        return self.losses
    
    def get_wins(self):
        return self.wins
    
    def get_record(self):
        return self.record
    
    def set_num(self, num):
        self.num = num
        
    def get_num(self):
        return self.num

In [3]:
# Game data from Pro Football Reference
games = {}
games[2020] = pd.read_csv('gamedata/games_2020.txt')
games[2019] = pd.read_csv('gamedata/games_2019.txt')
games[2018] = pd.read_csv('gamedata/games_2018.txt')
games[2017] = pd.read_csv('gamedata/games_2017.txt')
games[2016] = pd.read_csv('gamedata/games_2016.txt')

In [4]:
# Change this year value to try out different years. 
# I have only collected data from 2016 onwards, but it's readily available on Pro Football Reference.
year = 2020
name_to_abbrev = {}

# NFC East
if year <= 2019:
    WAS = Team('Washington Redskins', year = year)
    name_to_abbrev['Washington Redskins'] = WAS
else:
    WFT = Team('Washington Football Team', year = year)                
    name_to_abbrev['Washington Football Team'] = WFT
NYG = Team('New York Giants', year = year)
name_to_abbrev['New York Giants'] = NYG
DAL = Team('Dallas Cowboys', year = year)
name_to_abbrev['Dallas Cowboys'] = DAL
PHI = Team('Philadelphia Eagles', year = year)
name_to_abbrev['Philadelphia Eagles'] = PHI

# NFC South
NO = Team('New Orleans Saints', year = year)
name_to_abbrev['New Orleans Saints'] = NO
TB = Team('Tampa Bay Buccaneers', year = year)
name_to_abbrev['Tampa Bay Buccaneers'] = TB
ATL = Team('Atlanta Falcons', year = year)
name_to_abbrev['Atlanta Falcons']= ATL
CAR = Team('Carolina Panthers', year = year)
name_to_abbrev['Carolina Panthers'] = CAR

# NFC West
SEA = Team('Seattle Seahawks', year = year)
name_to_abbrev['Seattle Seahawks'] = SEA
LAR = Team('Los Angeles Rams', year = year)
name_to_abbrev['Los Angeles Rams'] = LAR
ARI = Team('Arizona Cardinals', year = year)
name_to_abbrev['Arizona Cardinals'] = ARI
SF = Team('San Francisco 49ers', year = year)
name_to_abbrev['San Francisco 49ers'] = SF

# NFC North
GB = Team('Green Bay Packers', year = year)
name_to_abbrev['Green Bay Packers'] = GB 
CHI = Team('Chicago Bears', year = year)
name_to_abbrev['Chicago Bears'] = CHI 
MIN = Team('Minnesota Vikings', year = year)
name_to_abbrev['Minnesota Vikings'] = MIN 
DET = Team('Detroit Lions', year = year)
name_to_abbrev['Detroit Lions'] = DET 

# AFC East
BUF = Team('Buffalo Bills', year = year)
name_to_abbrev['Buffalo Bills'] = BUF 
MIA = Team('Miami Dolphins', year = year)
name_to_abbrev['Miami Dolphins'] = MIA 
NE = Team('New England Patriots', year = year)
name_to_abbrev['New England Patriots'] = NE 
NYJ = Team('New York Jets', year = year)
name_to_abbrev['New York Jets'] = NYJ 

# AFC South
TEN = Team('Tennessee Titans', year = year)
name_to_abbrev['Tennessee Titans'] = TEN 
IND = Team('Indianapolis Colts', year = year)
name_to_abbrev['Indianapolis Colts'] = IND 
HOU = Team('Houston Texans', year = year)
name_to_abbrev['Houston Texans'] = HOU 
JAX = Team('Jacksonville Jaguars', year = year)
name_to_abbrev['Jacksonville Jaguars'] = JAX 

# AFC West
KC = Team('Kansas City Chiefs', year = year)
name_to_abbrev['Kansas City Chiefs'] = KC 
if year <= 2019:
    OAK = Team('Oakland Raiders', year = year)                  
    name_to_abbrev['Oakland Raiders'] = OAK 
else:
    LV = Team('Las Vegas Raiders', year = year)                  
    name_to_abbrev['Las Vegas Raiders'] = LV 
DEN = Team('Denver Broncos', year = year)
name_to_abbrev['Denver Broncos'] = DEN 
if year <= 2016:
    SD = Team('San Diego Chargers', year = year)             
    name_to_abbrev['San Diego Chargers'] = SD
else:
    LAC = Team('Los Angeles Chargers', year = year)             
    name_to_abbrev['Los Angeles Chargers'] = LAC

# AFC North
PIT = Team('Pittsburgh Steelers', year = year)
name_to_abbrev['Pittsburgh Steelers'] = PIT
BAL = Team('Baltimore Ravens', year = year)
name_to_abbrev['Baltimore Ravens'] = BAL 
CLE = Team('Cleveland Browns', year = year)
name_to_abbrev['Cleveland Browns'] = CLE 
CIN = Team('Cincinnati Bengals', year = year)
name_to_abbrev['Cincinnati Bengals'] = CIN

In [5]:
def lombardi_num(team_start, return_levels = False):
    done = []
    queue = [team_start]
    queue_next = []
    levels = (set(queue), )
    level = 0
    if not return_levels:
        print(f'Level 0: {set(queue)}\n')
    while queue:
        team = queue[0]
        if team in done:
            queue.pop(0)
            continue
        root = name_to_abbrev[team]
        root.set_num(level)
        for loss_team in set(root.get_losses()):
            child = name_to_abbrev[loss_team]
            if child.get_name() in done:
                continue
            elif child.get_name() in queue:
                continue
            elif child.get_name() in queue_next:
                continue
            else:
                queue_next.append(child.get_name())
        queue.pop(0)
        done.append(team)
        if len(queue) == 0:
            queue = queue_next
            queue_next = []
            level += 1
            if len(queue) != 0:
                if return_levels:
                    levels = levels + (set(queue), )
                else:
                    print(f'Level {level}: {set(queue)}\n')
    if len(done) != 32:
        print(f'Only {len(done)} team(s) traversed!')
    if return_levels:
        return levels
    return

In [6]:
def jackson_num(team_start, return_levels = False):
    done = []
    queue = [team_start]
    queue_next = []
    levels = (set(queue), )
    level = 0
    if not return_levels:
        print(f'Level 0: {set(queue)}\n')
    while queue:
        team = queue[0]
        if team in done:
            queue.pop(0)
            continue
        root = name_to_abbrev[team]
        root.set_num(level)
        for win_team in set(root.get_wins()):
            child = name_to_abbrev[win_team]
            if child.get_name() in done:
                continue
            elif child.get_name() in queue:
                continue
            elif child.get_name() in queue_next:
                continue
            else:
                queue_next.append(child.get_name())
        queue.pop(0)
        done.append(team)
        if len(queue) == 0:
            queue = queue_next
            queue_next = []
            level += 1
            if len(queue) != 0:
                if return_levels:
                    levels = levels + (set(queue), )
                else:
                    print(f'Level {level}: {set(queue)}\n')
    if len(done) != 32:
        print(f'Only {len(done)} team(s) traversed!')
    if return_levels:
        return levels
    return

In [7]:
# How far is your team removed from losing to the worst team in the league? (regular season games only)
jackson_num('Jacksonville Jaguars')

Level 0: {'Jacksonville Jaguars'}

Level 1: {'Indianapolis Colts'}

Level 2: {'Las Vegas Raiders', 'Green Bay Packers', 'Houston Texans', 'Cincinnati Bengals', 'Detroit Lions', 'Minnesota Vikings', 'New York Jets', 'Chicago Bears', 'Tennessee Titans'}

Level 3: {'Baltimore Ravens', 'New England Patriots', 'Kansas City Chiefs', 'Pittsburgh Steelers', 'Denver Broncos', 'Cleveland Browns', 'Philadelphia Eagles', 'San Francisco 49ers', 'New Orleans Saints', 'Los Angeles Chargers', 'Washington Football Team', 'Carolina Panthers', 'Arizona Cardinals', 'New York Giants', 'Buffalo Bills', 'Los Angeles Rams', 'Tampa Bay Buccaneers', 'Atlanta Falcons'}

Level 4: {'Miami Dolphins', 'Dallas Cowboys', 'Seattle Seahawks'}



In [8]:
# How far is your team removed from beating the best team in the league? (regular season games only)
# I'm assuming KC wins for now lol
lombardi_num('Kansas City Chiefs')

Level 0: {'Kansas City Chiefs'}

Level 1: {'Los Angeles Chargers', 'Las Vegas Raiders'}

Level 2: {'New England Patriots', 'Miami Dolphins', 'Denver Broncos', 'New Orleans Saints', 'Carolina Panthers', 'Buffalo Bills', 'Tampa Bay Buccaneers', 'Atlanta Falcons', 'Indianapolis Colts'}

Level 3: {'Baltimore Ravens', 'Houston Texans', 'Green Bay Packers', 'Pittsburgh Steelers', 'Seattle Seahawks', 'Cleveland Browns', 'Philadelphia Eagles', 'San Francisco 49ers', 'Arizona Cardinals', 'Minnesota Vikings', 'Detroit Lions', 'Los Angeles Rams', 'Chicago Bears', 'Jacksonville Jaguars', 'Tennessee Titans', 'Dallas Cowboys'}

Level 4: {'New York Giants', 'Cincinnati Bengals', 'Washington Football Team', 'New York Jets'}

